<a href="https://colab.research.google.com/github/Dkepffl/Dacon/blob/main/HD_AI_Challenge/HD_FE_1025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HD 현대 AI Chalenge**
- 구글 드라이브 어마운트를 통해 데이터 로드
- 10.22 기준 리더보드 최저 점수(45.6250669998) 코드


## **| 구글 드라이브 어마운트**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **| 필요한 라이브러리 로드**

In [ ]:
# import libraries
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os

## **| 작업 폴더 변경 및 데이터 로드**

In [ ]:
os.chdir("/content/drive/MyDrive/HD현대 AI Challenge") # 각자 드라이브에 맞춰 변경
os.getcwd()

'/content/drive/MyDrive/HD현대 AI Challenge'

In [ ]:
train = pd.read_csv('Data/train.csv').drop('SAMPLE_ID', axis=1)
test = pd.read_csv('Data/test.csv').drop('SAMPLE_ID', axis=1)

In [ ]:
# 원본 데이터셋 별도 저장
train_proto = train.copy()
test_proto = test.copy()

## **| 데이터 전처리**

### **1. ATA 변수 처리**

In [ ]:
from datetime import datetime

# datetime 컬럼 처리
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train, test]:
  df['YEAR'] = df['ATA'].dt.year
  df['MONTH'] = df['ATA'].dt.month
  df['DAY'] = df['ATA'].dt.day
  df['HOUR'] = df['ATA'].dt.hour
  df['MINUTE'] = df['ATA'].dt.minute
  df['WEEKDAY'] = df['ATA'].dt.weekday

In [ ]:
train.drop('ATA', axis=1, inplace=True)
test.drop('ATA', axis=1, inplace=True)

### **2. 파생 변수 추가**

#### **선박 유형 별 선박 크기 관련 컬럼 추가**

In [ ]:
temp_train=train.copy()
temp_test=test.copy()

temp_train['SIZE']= temp_train['GT']+temp_train['BREADTH']+temp_train['DEADWEIGHT']+temp_train['DEPTH']+temp_train['DRAUGHT']+temp_train['LENGTH']
temp_test['SIZE']= temp_test['GT']+temp_test['BREADTH']+temp_test['DEADWEIGHT']+temp_test['DEPTH']+temp_test['DRAUGHT']+temp_test['LENGTH']

In [ ]:
ship_type1=train['SHIP_TYPE_CATEGORY'].unique().tolist()
ship_type2=test['SHIP_TYPE_CATEGORY'].unique().tolist()

In [ ]:
SHIP_dist_train=temp_train.groupby('SHIP_TYPE_CATEGORY')['SIZE'].agg(['mean','std']).reset_index()
SHIP_dist_test=temp_test.groupby('SHIP_TYPE_CATEGORY')['SIZE'].agg(['mean','std']).reset_index()

In [ ]:
for ship in ship_type1:
  train.loc[train['SHIP_TYPE_CATEGORY']==ship,'SIZE_MEAN'] = SHIP_dist_train.loc[SHIP_dist_train['SHIP_TYPE_CATEGORY']==ship,'mean'].values[0]
  train.loc[train['SHIP_TYPE_CATEGORY']==ship,'SIZE_STD'] = SHIP_dist_train.loc[SHIP_dist_train['SHIP_TYPE_CATEGORY']==ship,'std'].values[0]

for ship in ship_type2:
  test.loc[test['SHIP_TYPE_CATEGORY']==ship,'SIZE_MEAN'] = SHIP_dist_test.loc[SHIP_dist_test['SHIP_TYPE_CATEGORY']==ship,'mean'].values[0]
  test.loc[test['SHIP_TYPE_CATEGORY']==ship,'SIZE_STD'] = SHIP_dist_test.loc[SHIP_dist_test['SHIP_TYPE_CATEGORY']==ship,'std'].values[0]

#### **ARI_PO 별 DIST 통계량**

In [ ]:
port_list1=train['ARI_PO'].unique().tolist()
port_list2=test['ARI_PO'].unique().tolist()

In [ ]:
DIST_dist_train=train.groupby('ARI_PO')['DIST'].agg(['mean','std']).reset_index()
DIST_dist_test=train.groupby('ARI_PO')['DIST'].agg(['mean','std']).reset_index()

In [ ]:
for port in port_list1:
  train.loc[train['ARI_PO']==port,'DIST_MEAN'] = DIST_dist_train.loc[DIST_dist_train['ARI_PO']==port,'mean'].values[0]
  train.loc[train['ARI_PO']==port,'DIST_STD'] = DIST_dist_train.loc[DIST_dist_train['ARI_PO']==port,'std'].values[0]

for port in port_list2:
  test.loc[test['ARI_PO']==port,'DIST_MEAN'] = DIST_dist_test.loc[DIST_dist_test['ARI_PO']==port,'mean'].values[0]
  test.loc[test['ARI_PO']==port,'DIST_STD'] = DIST_dist_test.loc[DIST_dist_test['ARI_PO']==port,'std'].values[0]

#### **ARI_CO 별 LENGTH 통계량**

In [ ]:
co_list1=train['ARI_CO'].unique().tolist()
co_list2=test['ARI_CO'].unique().tolist()

In [ ]:
LENGTH_length_train=train.groupby('ARI_CO')['LENGTH'].agg(['mean','std']).reset_index()
LENGTH_length_test=train.groupby('ARI_CO')['LENGTH'].agg(['mean','std']).reset_index()

In [ ]:
for co in co_list1:
  train.loc[train['ARI_CO']==co,'LENGTH_MEAN'] = LENGTH_length_train.loc[LENGTH_length_train['ARI_CO']==co,'mean'].values[0]
  train.loc[train['ARI_CO']==co,'LENGTH_STD'] = LENGTH_length_train.loc[LENGTH_length_train['ARI_CO']==co,'std'].values[0]

for co in co_list2:
  test.loc[test['ARI_CO']==co,'LENGTH_MEAN'] = LENGTH_length_test.loc[LENGTH_length_test['ARI_CO']==co,'mean'].values[0]
  test.loc[test['ARI_CO']==co,'LENGTH_STD'] = LENGTH_length_test.loc[LENGTH_length_test['ARI_CO']==co,'std'].values[0]

### **3. 카테고리 변수 라벨 인코딩**

In [ ]:
from sklearn.preprocessing import LabelEncoder
import bisect
from tqdm import tqdm

categorical_features = ['ARI_CO','ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER','FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
  le = LabelEncoder()
  train[feature] = le.fit_transform(train[feature].astype(str))
  le_classes_set = set(le.classes_)
  test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
  le_classes = le.classes_.tolist()
  bisect.insort_left(le_classes, '-1')
  le.classes_ = np.array(le_classes)
  test[feature] = le.transform(test[feature].astype(str))
  encoders[feature] = le

Encoding features: 100%|██████████| 6/6 [00:02<00:00,  2.05it/s]


### **4. 미사용 컬럼 제거**

In [ ]:
train.drop(columns=['ID', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BUILT','BN','DAY'], inplace=True)
test.drop(columns=['ID', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BUILT','BN','DAY'], inplace=True)

In [ ]:
co_matrix=train.corr()
correlations_with_x = co_matrix['CI_HOUR']
correlations_with_x.abs().sort_values(ascending=False)

CI_HOUR               1.000000
DIST                  0.189145
WEEKDAY               0.111632
SHIP_TYPE_CATEGORY    0.087101
LENGTH_MEAN           0.084063
DEADWEIGHT            0.077954
GT                    0.059661
DEPTH                 0.051760
BREADTH               0.043711
DRAUGHT               0.040309
SIZE_MEAN             0.040032
LENGTH                0.035334
DIST_STD              0.028731
YEAR                  0.021253
PORT_SIZE             0.020635
SIZE_STD              0.013462
LENGTH_STD            0.008949
MINUTE                0.008505
DIST_MEAN             0.007737
HOUR                  0.007474
ARI_CO                0.005224
MONTH                 0.004286
ARI_PO                0.000459
ATA_LT                0.000090
Name: CI_HOUR, dtype: float64

### **5. 결측치 처리**

In [ ]:
train = train.fillna(-1)
test = test.fillna(-1)

### **Autoencoder**

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 모델 구성
input_dim = len(train.drop('CI_HOUR',axis=1).columns)  # 입력 데이터의 차원

encoding_dim = 2  # 잠재 표현의 차원
input_layer = keras.Input(shape=(input_dim,))
encoder = layers.Dense(encoding_dim, activation='relu')(input_layer)
decoder = layers.Dense(input_dim, activation='sigmoid')(encoder)

autoencoder = keras.Model(input_layer, decoder)

# 모델 컴파일
autoencoder.compile(optimizer='adam', loss='mean_absolute_error')

# 데이터프레임을 Numpy 배열로 변환
data_array = train.drop('CI_HOUR',axis=1).values

# 모델 학습
autoencoder.fit(data_array, data_array, epochs=50, batch_size=32)

# 학습된 모델을 사용하여 잠재 표현 얻기
encoder_model = keras.Model(input_layer, encoder)
latent_representation = encoder_model.predict(data_array)

print("잠재 표현의 형태:", latent_representation.shape)

Epoch 1/50
12249/12249 [==============================] - 51s 4ms/step - loss: 13017.7217
Epoch 2/50
12249/12249 [==============================] - 29s 2ms/step - loss: 13017.6846
Epoch 3/50
12249/12249 [==============================] - 29s 2ms/step - loss: 13017.7412
Epoch 4/50
12249/12249 [==============================] - 30s 2ms/step - loss: 13017.7334
Epoch 5/50
12249/12249 [==============================] - 29s 2ms/step - loss: 13017.7021
Epoch 6/50
12249/12249 [==============================] - 29s 2ms/step - loss: 13017.6807
Epoch 7/50
12249/12249 [==============================] - 32s 3ms/step - loss: 13017.7422
Epoch 8/50
12249/12249 [==============================] - 30s 2ms/step - loss: 13017.6670
Epoch 9/50
12249/12249 [==============================] - 29s 2ms/step - loss: 13017.7002
Epoch 10/50
12249/12249 [==============================] - 29s 2ms/step - loss: 13017.7373
Epoch 11/50
12249/12249 [==============================] - 30s 2ms/step - loss: 13017.7109
Epoch 12

In [ ]:
# 잠재 표현을 데이터프레임에 추가
latent_df = pd.DataFrame(latent_representation, columns=['LA1','LA2'])

In [ ]:
train = pd.concat([train, latent_df], axis=1)

In [ ]:
# 학습된 모델을 사용하여 잠재 표현 얻기
latent_2 = encoder_model.predict(test.values)
latent_df2 = pd.DataFrame(latent_2, columns=['LA1','LA2'])
test = pd.concat([test, latent_df2], axis=1)

6891/6891 [==============================] - 20s 3ms/step


In [ ]:
test.shape

(220491, 25)

### **6. 스케일링**

In [ ]:
from sklearn.preprocessing import StandardScaler

scl = StandardScaler()

train_scld = scl.fit_transform(train.drop('CI_HOUR', axis=1))
train_scld = pd.DataFrame(train_scld, columns=train.drop('CI_HOUR', axis=1).columns)

In [ ]:
test_scld = scl.transform(test)
test_scld = pd.DataFrame(test_scld, columns=test.columns)

### **학습/검증 데이터셋 생성**

In [ ]:
from sklearn.model_selection import train_test_split
X = train_scld
Y = train["CI_HOUR"]
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=42)

### **Catboost 모델링**

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.0 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor

cbc = CatBoostRegressor(verbose=1000, iterations=20000, objective='MAE', use_best_model=True)
cbc.fit(x_train, y_train, eval_set=(x_valid, y_valid))

0:	learn: 60.8227174	test: 59.7591444	best: 59.7591444 (0)	total: 175ms	remaining: 58m 12s
1000:	learn: 46.4171379	test: 45.6045622	best: 45.6045622 (1000)	total: 1m 52s	remaining: 35m 35s
2000:	learn: 45.3428702	test: 44.6519475	best: 44.6519475 (2000)	total: 3m 26s	remaining: 30m 56s
3000:	learn: 45.1065482	test: 44.4828876	best: 44.4823510 (2941)	total: 4m 59s	remaining: 28m 18s
4000:	learn: 45.1029860	test: 44.4801686	best: 44.4801574 (3990)	total: 6m 35s	remaining: 26m 22s
5000:	learn: 45.1012531	test: 44.4790336	best: 44.4790333 (4999)	total: 8m 9s	remaining: 24m 26s
6000:	learn: 45.0983534	test: 44.4768215	best: 44.4768132 (5997)	total: 9m 45s	remaining: 22m 45s
7000:	learn: 45.0936981	test: 44.4715044	best: 44.4715007 (6997)	total: 11m 21s	remaining: 21m 5s
8000:	learn: 45.0812713	test: 44.4663204	best: 44.4663204 (8000)	total: 12m 58s	remaining: 19m 27s
9000:	learn: 45.0780907	test: 44.4654182	best: 44.4654182 (9000)	total: 14m 31s	remaining: 17m 45s
10000:	learn: 45.0495647	t

In [ ]:
importances = cbc.get_feature_importance(type='PredictionValuesChange')
feature_importances = pd.Series(importances, index=X.columns).sort_values()

In [ ]:
feature_importances

LA2                    0.000000
MINUTE                 0.038952
ATA_LT                 0.138841
DRAUGHT                0.152143
DEPTH                  0.167039
MONTH                  0.172575
HOUR                   0.759179
SIZE_STD               0.866435
SIZE_MEAN              1.739093
YEAR                   1.969992
ARI_PO                 2.337443
DIST_MEAN              2.376453
BREADTH                2.567632
LENGTH_MEAN            2.638245
DEADWEIGHT             3.455965
GT                     3.494014
WEEKDAY                4.180134
ARI_CO                 4.421164
DIST_STD               4.661720
SHIP_TYPE_CATEGORY     5.044085
LENGTH_STD             5.116776
PORT_SIZE              7.478180
LENGTH                 7.954822
LA1                    9.617012
DIST                  28.652104
dtype: float64

In [ ]:
from sklearn.metrics import mean_absolute_error

pred = cbc.predict(x_valid)
mean_absolute_error(pred, y_valid)

44.44156325524698

In [ ]:
# test 데이터셋에 대한 예측
final = cbc.predict(test_scld)

In [ ]:
final.shape

(220491,)

## **| 제출 파일 생성 및 저장**

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/HD현대 AI Challenge/Data/sample_submission.csv')
submit['CI_HOUR'] = final

In [ ]:
submit.to_csv('YOUN/sub_FE1025(4).csv', index=False) # 파일 이름 수정